In [1]:
import pandas as pd
import numpy as np
import os

In [12]:
rmaps.columns

Index([u'ID', u'GeneID', u'geneSymbol', u'chr', u'strand', u'exonStart_0base',
       u'exonEnd', u'upstreamES', u'upstreamEE', u'downstreamES',
       u'downstreamEE', u'ID.1', u'IJC_SAMPLE_1', u'SJC_SAMPLE_1',
       u'IJC_SAMPLE_2', u'SJC_SAMPLE_2', u'IncFormLen', u'SkipFormLen',
       u'PValue', u'FDR', u'IncLevel1', u'IncLevel2', u'IncLevelDifference'],
      dtype='object')

In [39]:
rmats = pd.read_table('/home/gpratt/projects/encode/analysis/ad-hoc/rMATS/ENCSR098NHI_vs_ENCSR661HEL/MATS_output/SE.MATS.JunctionCountOnly.txt')
fdr = 0.1
direction = "both"
inc_level = 0.1

inc_level = abs(inc_level) # clear up confusion about included/excludedness

if(direction=="both"):
    rmats = rmats[(abs(rmats['IncLevelDifference']) > inc_level) & \
                                      (rmats['FDR'] <= fdr)]
elif(direction=="included"):
    rmats = rmats[(rmats['IncLevelDifference'] > inc_level) & \
                                      (rmats['FDR'] <= fdr)]
elif(direction=="excluded"):
    rmats = rmats[(rmats['IncLevelDifference'] < -inc_level) & \
                                      (rmats['FDR'] <= fdr)]
else:
    print("Warning: direction undefined, returning all IncLevels of FDR < {}".format(fdr))
    rmats = rmats[rmats['FDR'] <= fdr]

In [40]:
def rmats_to_miso(row):
    if row['strand'] == '+':
        return '{}:{}:{}:{}@{}:{}:{}:{}@{}:{}:{}:{}\t{}'.format(
            row['chr'],row['upstreamES'],row['upstreamEE'],row['strand'],
            row['chr'],row['exonStart_0base'],row['exonEnd'],row['strand'],
            row['chr'],row['downstreamES'],row['downstreamEE'],row['strand'],
            row['GeneID']
        )
    else:
        return '{}:{}:{}:{}@{}:{}:{}:{}@{}:{}:{}:{}\t{}'.format(
            row['chr'],row['downstreamES'],row['downstreamEE'],row['strand'],
            row['chr'],row['exonStart_0base'],row['exonEnd'],row['strand'],
            row['chr'],row['upstreamES'],row['upstreamEE'],row['strand'],
            row['GeneID']
        )

In [41]:
rmats['miso'] = rmats.apply(rmats_to_miso, axis=1)


In [44]:
pd.concat([rmats['miso'],rmats['GeneID']],axis=1)

,miso,GeneID
0,chr19:29097594:29098011:+@chr19:29101166:29101...,ENSG00000266976.1
1,chr5:138629336:138629494:+@chr5:138642927:1386...,ENSG00000015479.13
2,chr17:30692348:30692506:+@chr17:30693683:30693...,ENSG00000010244.12
3,chr11:70266505:70266616:+@chr11:70267575:70267...,ENSG00000085733.11
5,chr1:111992458:111992495:+@chr1:111996842:1119...,ENSG00000116459.6
6,chr6:168311963:168312169:+@chr6:168314082:1683...,ENSG00000130396.16
7,chr12:112451119:112451413:+@chr12:112457573:11...,ENSG00000089248.6
8,chr12:112451119:112451413:+@chr12:112457559:11...,ENSG00000089248.6
9,chr1:46072153:46072263:+@chr1:46072992:4607400...,ENSG00000132780.12
10,chr17:79977733:79977785:-@chr17:79977516:79977...,ENSG00000169689.10


In [51]:
def generate_rmats_as_miso(manifest_file,
                           rmats_dir,
                           uid,
                           fdr=0.1,
                           inc_level=0,
                           direction="both"):
    """
    manifest: /home/gpratt/Dropbox/encode_integration/20160408_ENCODE_MASTER_ID_LIST_AllDatasets.csv
    uid: 204
    rep: 1
    """
    directory = ""
    df = pd.read_table(manifest_file,dtype={'uID':str})
    try:
        control = list(df[df['uID']==str(uid)]['RNASEQ_ControlENC'])[0]
        rbp = list(df[df['uID']==str(uid)]['RNASEQ_ENCODEAccID'])[0]
        directory = os.path.join(rmats_dir,rbp+'_vs_'+control)
        print(directory)
        tsv_filestring = os.path.join(directory,'MATS_output/SE.MATS.JunctionCountOnly.txt')
        rmats = pd.read_table(tsv_filestring,sep="\t")
        inc_level = abs(inc_level) # clear up confusion about included/excludedness

        if(direction=="both"):
            rmats = rmats[(abs(rmats['IncLevelDifference']) > inc_level) & \
                                              (rmats['FDR'] <= fdr)]
        elif(direction=="included"):
            rmats = rmats[(rmats['IncLevelDifference'] > inc_level) & \
                                              (rmats['FDR'] <= fdr)]
        elif(direction=="excluded"):
            rmats = rmats[(rmats['IncLevelDifference'] < -inc_level) & \
                                              (rmats['FDR'] <= fdr)]
        else:
            print("Warning: direction undefined, returning all IncLevels of FDR < {}".format(fdr))
            rmats = rmats[rmats['FDR'] <= fdr]
            
        rmats['miso'] = rmats.apply(rmats_to_miso, axis=1)
        return pd.concat([rmats['miso'],rmats['GeneID']],axis=1)
    except Exception as e:
        print("Could not find the diffexp files for uid: {}".format(uid))
        print(e)

In [52]:
manifest_file = '/home/gpratt/Dropbox/encode_integration/20160408_ENCODE_MASTER_ID_LIST_AllDatasets.csv'
uid = 204
rmats_dir = '/home/gpratt/projects/encode/analysis/ad-hoc/rMATS/'
generate_rmats_as_miso(manifest_file,rmats_dir,uid)

/home/gpratt/projects/encode/analysis/ad-hoc/rMATS/ENCSR767LLP_vs_ENCSR104ABF


,miso,GeneID
0,chr5:140998364:140998566:-@chr5:140967790:1409...,ENSG00000131504.11
1,chr7:22999874:23000034:-@chr7:22986570:2298686...,ENSG00000122591.7
2,chr11:70266328:70266616:+@chr11:70267575:70267...,ENSG00000085733.11
3,chr11:70266505:70266616:+@chr11:70267575:70267...,ENSG00000085733.11
4,chr12:2975558:2975687:-@chr12:2974520:2974565:...,ENSG00000111206.8
5,chr13:76378424:76378677:+@chr13:76383289:76383...,ENSG00000136153.15
6,chr3:167759179:167759262:-@chr3:167758573:1677...,ENSG00000173905.4
7,chr17:27413455:27413595:-@chr17:27412621:27412...,ENSG00000221995.4
8,chr1:29379615:29379824:+@chr1:29386933:2938699...,ENSG00000159023.14
9,chr1:225695652:225695719:-@chr1:225692692:2256...,ENSG00000154380.12
